# Importing Libraries

In [ ]:
# import libraries
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential
from tensorflow.math import confusion_matrix
from matplotlib.ticker import MultipleLocator
import random
from json import dump, load
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
import numpy as np 
import pandas as pd
import os, re, time, math, tqdm, itertools
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.offline as pyo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
import keras
from keras.layers import Conv2D, Conv1D, MaxPooling2D, MaxPooling1D, Flatten, BatchNormalization, Dense
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.callbacks import CSVLogger, ModelCheckpoint
plt.rcParams.update({'font.family':'Nimbus Roman'})

# Initializing Constants

In [ ]:
NUM_ROUNDS = 15
CLIENT_RATIO = 0.3
TOTAL_CLIENTS = 100
NUM_CLIENTS = int(CLIENT_RATIO * TOTAL_CLIENTS)
INIT_THRESHOLD = 4.0
EXPECTED_RESPONSE = 0.8

# Importing the Dataset

In [ ]:
network_data = pd.read_csv('/content/02-14-2018.csv')


In [ ]:
# drop null or missing columns
cleaned_data = network_data.dropna()
cleaned_data.isna().sum().to_numpy()
# encode the column labels
label_encoder = LabelEncoder()
cleaned_data['Label']= label_encoder.fit_transform(cleaned_data['Label'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# make 3 seperate datasets for 3 feature labels
data_1 = cleaned_data[cleaned_data['Label'] == 0]
data_2 = cleaned_data[cleaned_data['Label'] == 1]
data_3 = cleaned_data[cleaned_data['Label'] == 2]

# make benign feature
y_1 = np.zeros(data_1.shape[0])
y_benign = pd.DataFrame(y_1)

# make bruteforce feature
y_2 = np.ones(data_2.shape[0])
y_bf = pd.DataFrame(y_2)

# make bruteforceSSH feature
y_3 = np.full(data_3.shape[0], 2)
y_ssh = pd.DataFrame(y_3)

# merging the original dataframe
X = pd.concat([data_1, data_2, data_3], sort=True)
y = pd.concat([y_benign, y_bf, y_ssh], sort=True)
train_dataset = pd.concat([data_1, data_2, data_3])


# L labelled dataset, U unlabelled dataset

In [ ]:
test_dataset = train_dataset.sample(frac=0.1)
train_dataset, U_unlabelled_dataset = train_test_split(train_dataset, test_size=0.8)

target_train = train_dataset['Label']
target_test = test_dataset['Label']

# Distributing the dataset among the clients

In [ ]:
div_list = [np.random.randint(3000,6000) for i in range(NUM_CLIENTS)]
origin_list = [np.random.randint(0,125973-6000) for i in range(NUM_CLIENTS)]

In [ ]:
def prepare_for_training(train_dataset):
  target_train = train_dataset['Label']
  Y_train = to_categorical(target_train, num_classes=3)
  train_dataset = train_dataset.drop(columns = ["Timestamp", "Protocol","PSH Flag Cnt","Init Fwd Win Byts","Flow Byts/s","Flow Pkts/s", "Label"], axis=1)
  # making train & test splits
  X_train = train_dataset.values
  # reshape the data for CNN
  X_train = X_train.reshape(len(X_train), X_train.shape[1], 1)
  client_train_x = []
  client_train_y = []

  for i in range(NUM_CLIENTS):
    client_train_x.append(X_train[origin_list[i]:origin_list[i]+div_list[i]])
    client_train_y.append(Y_train[origin_list[i]:origin_list[i]+div_list[i]])
  return X_train,Y_train,client_train_x,client_train_y

# # Step 3: Select samples from U using a query function Q,

In [ ]:
threshold=len(U_unlabelled_dataset)
def new_labelled_dataset(U_unlabelled_dataset,model,step,per=0.4):

  X_train = U_unlabelled_dataset.drop(columns = ["Timestamp", "Protocol","PSH Flag Cnt","Init Fwd Win Byts","Flow Byts/s","Flow Pkts/s", "Label"], axis=1)

  df =U_unlabelled_dataset.copy()

  df["score"]=  model.predict(X_train).max(axis=1)


  
  df=df.sort_values(by='score', ascending=False)
  new_U_labelled_dataset=df.head(int(threshold*per)).drop(columns = ["score"], axis=1)

  U_unlabelled_dataset = df.tail(threshold-int(threshold*per*step)).drop(columns = ["score"], axis=1)
  
  return U_unlabelled_dataset,new_U_labelled_dataset


# Step 4: request the labels for the samples selected in step 3 from the expert A,

#### we don't need this step because we have all the labels

# Step 5: remove the selected samples from the dataset U and add the selected samples to L

In [ ]:
def update_L_labelled_dataset(new_labelled_dataset):
  return pd.concat([train_dataset , new_labelled_dataset])

# Model Utilities

In [ ]:
def create_server_model():
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=6, activation='relu', 
                    padding='same', input_shape=(73, 1)))
    model.add(BatchNormalization())
    
    # adding a pooling layer
    model.add(MaxPooling1D(pool_size=(3), strides=2, padding='same'))
    
    model.add(Conv1D(filters=64, kernel_size=6, activation='relu', 
                    padding='same', input_shape=(73, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=(3), strides=2, padding='same'))
    
    model.add(Conv1D(filters=64, kernel_size=6, activation='relu', 
                    padding='same', input_shape=(73, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=(3), strides=2, padding='same'))
    
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def model_cloner(model, learning_rate, optimizer):
    new_model = tf.keras.models.clone_model(model)
    new_model.set_weights(model.get_weights())
    if optimizer=='adam':
        new_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return new_model

In [ ]:
def train_client(num, model, lr,client_train_x,client_train_y):

  new_model = model_cloner(model, lr, 'adam')
  hist = new_model.fit(client_train_x[num], client_train_y[num], epochs=1, batch_size=2)

  return new_model, lr, round(hist.history['loss'][-1], 4)

# Federated active Learning

#Step 6: retain the model using the dataset L

In [ ]:
client_models = list(np.zeros((NUM_CLIENTS)))
arr = list()
arr1 = list()
server_model_norm = create_server_model()
server_model_norm.compile(optimizer = tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
serverhist1={
    "loss":[],
    "accuracy":[],
    "time taken":[]
}
step = 0
while U_unlabelled_dataset.empty==False:
    print( "Start of step : " + str(step))
    
      # Update training data
    if step != 0 :
      print( "update unlabled dataset of step : " + str(step))
      U_unlabelled_dataset ,new_labelled_dateset= new_labelled_dataset(U_unlabelled_dataset,server_model_norm,step)
      train_dataset = update_L_labelled_dataset(new_labelled_dateset)
    else : pass
    X_train,Y_train,client_train_x,client_train_y= prepare_for_training(train_dataset)
    print( "U unlabelled dataset: " + str(len(U_unlabelled_dataset)))

    for i in range(NUM_ROUNDS):
      print("-----"+str(i)+"---------")
      losses = []
      lr_init = []
      data= []
      serverhist1["time taken"].append(0)
      for j in range(NUM_CLIENTS):
        arr.append(time.time())
        data.append(train_client(j, server_model_norm, 0.01 ,client_train_x,client_train_y))

        client_models[j] = data[j][0]
        losses.append(data[j][2])
        lr_init.append(data[j][1])
        # time.sleep(2.519+random.random()*3-1.5)
        arr[-1] = time.time()-arr[-1]+2.519+random.random()*3-1.5
        serverhist1["time taken"][-1] = max(serverhist1["time taken"][-1], arr[-1])

      print(arr)
      # Aggregating model
      arr1.append(max(arr))
      sum1=[i*0 for i in client_models[0].get_weights()]
      for i in range(NUM_CLIENTS):
        sum1 = [i+j for i, j in zip(client_models[i].get_weights(), sum1)]
      server_model_norm.set_weights([i/NUM_CLIENTS for i in sum1])
      h=server_model_norm.evaluate(X_train[:5000, :],Y_train[:5000, :])
      serverhist1['loss'].append(h[0])
      serverhist1['accuracy'].append(h[1])
    print( "End of step : " + str(step))
    step+=1


print( "End of training")

Start of step : 0
U unlabelled dataset: 837039
-----0---------
 445/2744 [===>..........................] - ETA: 12s - loss: 0.8985 - accuracy: 0.7045

### Saving Training Metrics

In [ ]:
with open("/content/normal_out.json", "w") as f:
    dump(serverhist1, f, indent=4)

### Loading Training Metrics

In [ ]:
with open("/content/normal_out.json", "r") as f:
    serverhist1 = load(f)

### Time Taken per Round

In [ ]:
plt.plot(serverhist1["time taken"])
plt.ylabel("Time Taken")
plt.xlabel("Round Number")
plt.title("Time Taken per Round")
plt.grid()

plt.show()
plt.show()

### Performance of the server model after each round

In [ ]:
fig = plt.figure(dpi=300, figsize=(12, 6))
ax = fig.add_subplot(121)
ax.plot(serverhist1["accuracy"], color='green')
ax.grid()
ax.set_xlabel("Rounds")
ax.xaxis.set_minor_locator(MultipleLocator(1))
ax.set_ylabel("Accuracy")
ax.set_title("Accuracy")
ax = fig.add_subplot(122)
ax.plot(serverhist1["loss"], color='#FF4800')
ax.grid()
ax.set_xlabel("Rounds")
ax.xaxis.set_minor_locator(MultipleLocator(1))
ax.set_ylabel("Loss")
ax.set_title("Loss")
plt.tight_layout()
plt.show()

In [ ]:
conf_matrix = confusion_matrix(np.argmax(Y_train, axis=1), np.argmax(server_model_norm.predict(X_train), axis=1)).numpy()
fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(conf_matrix, cmap="Greens", alpha=0.6, vmin=-1000)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
ax.set_xticks(np.arange(3))
ax.set_yticks(np.arange(3))
ax.set_xticklabels(["Benign","bruteforce","bruteforceSSH"], size='x-large')
ax.set_yticklabels(["Benign","bruteforce","bruteforceSSH"], rotation=90, va="center", size='x-large')
plt.ylabel('Actuals', fontsize=18)
plt.title('Predictions', fontsize=18)
plt.tight_layout()
plt.show()